## U.S. Unemployment Rate Change by Metropolitan Statistical Area (MSA)

Created by Brian Dew (brianwdew@gmail.com), February 6, 2017.

The goal of this notebook is to create a U.S. map that shows how unemployment rates are changing in different local areas.

See https://www.bls.gov/lau/ for advance release information.

https://download.bls.gov/pub/time.series/la/la.areamaps (list of codes of all local areas)

http://gis.stackexchange.com/questions/198530/plotting-us-cities-on-a-map-with-matplotlib-and-basemap

http://stackoverflow.com/questions/39742305/how-to-use-basemap-python-to-plot-us-with-50-states

This folder on GitHub (https://github.com/bdecon/Python/Unemp_Map) contains four files needed to make the code work:
1. MSA.csv - contains the names, codes, and locations for metropolitan statistical areas
2. st99_d00.dbf - one of the files used for drawing state shapes
3. st99_d00.shp - main state shapefile
4. st99_d00.shx - final file for drawing state shapes

### Obtaining the city locations and their unemployment rate change values

First, we load the pandas and matplotlib libraries that will allow us to work with data and map it. The second step is to load both the data that do not change (city names, codes, and locations) and the data that do change (BLS unemployment rate estimates). The third step is to combine the two datasets.

In [1]:
# Import libraries and sign in to plotly
import pandas as pd
import config   # File that contains api key
import plotly.plotly as py
from plotly.graph_objs import *
py.sign_in('bdew', config.plotly_key)

In [2]:
# MSA.csv contains city names, their BLS codes (Metropolitan 
# MTAs/NECTAs only), and their latitude and longitude
MSA = pd.read_csv('MSA.csv')

# BLS flat file contains the unemployment rate by month since 2015
url = 'https://download.bls.gov/pub/time.series/la/la.data.0.CurrentU15-19'
df = pd.read_table(url, sep='\s+', na_values=['-'])

In [3]:
# Take only the unemployment data for MSAs and only monthly series 
df = df[df['series_id'].isin(MSA['fullcode']) & (df['period'] != 'M13')]
df.loc[:,'date'] = pd.to_datetime([f'{i[1]}-{i[2][1:]}-01' for i in df.values])

# Calculate the change over the same month, previous year
df['value_ch'] = df['value'].diff(periods=12)
df = df.groupby(['series_id']).tail(1)
df.rename(columns={'series_id':'fullcode'}, inplace=True)

# Take the date for use in chart title
date = df['date'].iloc[0].strftime('%B %Y')

# Combine the two datasets and show 2 rows to verify 
unemp_list = df.merge(MSA, on='fullcode')
unemp_list = unemp_list.reindex(columns=[*unemp_list.columns.tolist(), 'color', 'size'])

# Shorten MSA names for chart labels
unemp_list.loc[:,'MSA'] = unemp_list['area_code'].str.replace(
    'Metropolitan Statistical Area', 'MSA')

In [8]:
# From the change in unemployment rate, mark increases red, decreases green,
#   no change is gray. Circle size is absolute value of change times 10 plus 2. 
for i, r in unemp_list.iterrows():
    if r['value_ch'] > 0:
        values = ['#FF0000', abs(r[6])*10+2, f'▲{r[6]:0.1f}']
        unemp_list.at[i, ['color', 'size']] = values[0:2]
    if r['value_ch'] < 0:
        values = ['#32CD32', abs(r[6])*10+2, f'▼{r[6]:0.1f}']
        unemp_list.at[i, ['color', 'size']] = values[0:2]        
    if r['value_ch'] == 0:        
        values = ['#696969', 3, 'unchanged']
        unemp_list.at[i, ['color', 'size']] = values[0:2]         
    # Create a text column for plotly hover labels
    unemp_list.at[i,'text'] = f'{r[13]}<br>{date} Rate: {r[3]}% ({values[2]})'

### Save the results as an html table

In [9]:
# generate html table with data
table = unemp_list[['MSA', 'value', 'value_ch']]
table.columns = ['Metropolitan Statistical Area (MSA) Name', 'Unemp. rate', 'One-year change']
table = table.sort_values('One-year change').drop_duplicates() 
# Sortable css class allows for js script to sort each column
# https://www.kryogenix.org/code/browser/sorttable/
table.to_html('C:/Working/bdecon.github.io/Dash/unemp_table.html', border=0, 
              index=False, classes='sortable')

### Create a map with Plotly to show the changes

In [10]:
trace1 = {
  "geo": "geo", 
  "hoverinfo": "text", 
  "lat": unemp_list['latitude'], 
  "lon": unemp_list['longitude'], 
  "marker": {
    "color": unemp_list['color'], 
    "line": {
      "color": "#202020", 
      "width": 0.8
    }, 
    "opacity": 0.5, 
    "size": unemp_list['size'], 
    "symbol": "circle"
  }, 
  "mode": "markers", 
  "name": "Unemployment Rate Change by Metro Area", 
  "opacity": 1, 
  "showlegend": True, 
  "text": unemp_list['text'], 
  "type": "scattergeo", 
  "visible": True
}
data = Data([trace1])
layout = {
  "annotations": [
    {
      "x": 0.05, 
      "y": 0.925, 
      "align": "left", 
      "font": {"size": 14}, 
      "showarrow": False, 
      "text": "<b>Change in Unemployment Rate by Metropolitan Statistical Area (MSA)</b><br><i>Change over level in same month of previous year, as of {}</i>".format(date), 
      "xref": "x", 
      "yref": "y"
    }, 
    {
      "x": 0.05, 
      "y": 0.0725, 
      "align": "left", 
      "showarrow": False, 
      "text": "Hover to see details. Green is decrease in unemployment rate, red is increase. Circle size shows amount of change.  <br><b>Source:</b> U.S. Bureau of Labor Statistics, Local Area Unemployment Statistics", 
      "xref": "x", 
      "yref": "y"
    }
  ], 
  "autosize": True, 
  "dragmode": "zoom", 
  "geo": {
    "bgcolor": "#fff", 
    "countrycolor": "#808080", 
    "countrywidth": 0.7, 
    "domain": {
      "x": [0, 1.1], 
      "y": [0, 1.1]
    }, 
    "lataxis": {
      "dtick": 10, 
      "range": [20, 80], 
      "showgrid": False, 
      "tick0": 20
    }, 
    "lonaxis": {
      "dtick": 30, 
      "range": [-180, -50], 
      "showgrid": False, 
      "tick0": -180
    }, 
    "projection": {
      "scale": 1.03, 
      "type": "albers usa"
    }, 
    "resolution": 50, 
    "scope": "usa", 
    "showcountries": False, 
    "showlakes": False, 
    "showland": False, 
    "showrivers": False, 
    "showsubunits": True, 
    "subunitcolor": "#808080", 
    "subunitwidth": 0.7
  }, 
  "hidesources": False, 
  "hovermode": "closest", 
  "paper_bgcolor": "#fff", 
  "separators": ".,", 
  "showlegend": False, 
  "smith": False, 
  "xaxis": {
    "showgrid": False, 
    "zeroline": False
  }, 
  "yaxis": {
    "showgrid": False, 
    "zeroline": False
  },
  "margin": {
    "r": 0, 
    "t": 0, 
    "b": 0, 
    "l": 0
  },
}
fig = Figure(data=data, layout=layout)
plot_url = py.plot(fig, filename='MSA_Map')